# Provision AWS Resources to allow Kinesis Firehose to Stream Data to S3 Bucket

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
from glob import glob

import boto3
from dotenv import find_dotenv, load_dotenv

In [ ]:
%aimport src.s3.buckets
import src.s3.buckets as s3b

%aimport src.cw.cloudwatch_logs
import src.cw.cloudwatch_logs as cwlogs

%aimport src.iam.iam_roles
import src.iam.iam_roles as iamr

%aimport src.firehose.kinesis_firehose
import src.firehose.kinesis_firehose as knsfire

%aimport src.ec2.ec2_instances_sec_groups
import src.ec2.ec2_instances_sec_groups as ec2h

%aimport src.keypairs.ssh_keypairs
import src.keypairs.ssh_keypairs as ssh_keys

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
aws_region = os.getenv("AWS_REGION")

## About

In this notebook, the following AWS resources will be provisioned
- S3 bucket
- CloudWatch Logging group
- CloudWatch Logging stream
- IAM role
- Kinesis Firehose Delivery Stream

### Pre-Requisites
1. The following environment variables should be set with the user's AWS credendials ([1](https://docs.aws.amazon.com/sdk-for-php/v3/developer-guide/guide_credentials_environment.html), [2](https://docs.aws.amazon.com/sdk-for-php/v3/developer-guide/guide_credentials_profiles.html))
   - `AWS_ACCESS_KEY_ID`
   - `AWS_SECRET_KEY`
   - `AWS_REGION`

   These credentials must be associated to a user group whose users have been granted programmatic access to AWS resources. In order to configure this for an IAM user group, see the documentation [here](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html#id_users_create_console).

### Notes
1. All resources will be created in the same AWS region (specified by the environment variable `AWS_REGION`).

## User Inputs

In [ ]:
# S3
s3_bucket_name = "testwillz3s"

# IAM Role
iam_role_path = "/"
iam_role_name = "ec2-dummy-role"
iam_role_description = "BOTO3 ec2 dummy role"
iam_role_trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "",
            "Effect": "Allow",
            "Principal": {"Service": "firehose.amazonaws.com"},
            "Action": "sts:AssumeRole",
        }
    ],
}

# Kinesis Firehose Stream
stream_s3_destination_prefix = "datasets/twitter/kinesis-demo/YYYY/MM/dd/HH"
firehose_stream_name = "twitter_delivery_stream"

# CloudWatch Logging
cw_logs_group_name = f"kinesisfirehose_{firehose_stream_name}"

# EC2 Security Groups
sg_group_tags = [{"Key": "Name", "Value": "allow-inbound-ssh"}]

# SSH Key Pairs
key_fname = "aws_ec2_key"
keypair_name = "ec2-key-pair"

# EC2 Instances
ec2_instance_image_id = "ami-0cc00ed857256d2b4"
ec2_instance_type = "t2.micro"
ec2_instance_tags_list = [{"Key": "Name", "Value": "my-ec2-instance"}]

## Create the S3 Bucket

In [ ]:
%%time
s3_bucket_creation_response = s3b.create_s3_bucket(s3_bucket_name, aws_region)
s3_bucket_creation_response

## Create CloudWatch Log Group and Stream

In [ ]:
%%time
cw_log_creation_response, cw_stream_creation_response = cwlogs.create_cw_logs_group_stream(
    cw_logs_group_name, firehose_stream_name, aws_region
)
cw_log_creation_response

In [ ]:
cw_stream_creation_response

## Create IAM Role

In [ ]:
%%time
iam_role_creation_response = iamr.create_iam_role(
    iam_role_path,
    iam_role_name,
    iam_role_description,
    iam_role_trust_policy,
    aws_region,
)
iam_role_creation_response

## Create the Kinesis Firehose Delivery Stream

In [ ]:
%%time
kinesis_firehose_stream_creation_response = knsfire.create_kinesis_firehose_stream(
    firehose_stream_name,
    iam_role_creation_response['Role']["Arn"],
    s3_bucket_name,
    stream_s3_destination_prefix,
    cw_logs_group_name,
    iam_role_name,
    iam_role_path,
    aws_region,
)
kinesis_firehose_stream_creation_response

In [ ]:
knsfire.describe_kinesis_firehose_stream(firehose_stream_name, aws_region)

## EC2

### Create EC2 Security Group to Allow SSH Traffic on Port 22

In [ ]:
%%time
ec2h.create_security_group(
    "allow-inbound-ssh",
    "Allow inbound SSH traffic",
    aws_region,
    sg_group_tags,
)

### Create SSH Key Pair

Create a local SSH Key Pair file

In [ ]:
ssh_key_creation_response = ssh_keys.create_key_pair(
    keypair_name,
    aws_region,
    "/tmp",
    key_fname,
    [{"Key": "Name", "Value": "my-ssh-key-pair"}],
)
ssh_key_creation_response

Show the created keypair file

In [ ]:
glob(f"/tmp/{key_fname}*.pem")

### Create EC2 Instance

In [ ]:
%%time
created_instance_response = ec2h.create_instance(
    image_id=ec2_instance_image_id,
    instance_type=ec2_instance_type,
    keypair_name=keypair_name,
    region=aws_region,
    tags_list=ec2_instance_tags_list,
)
created_instance_response

### Attach EC2 Security Group to EC2 Instance To Grant SSH Access to Instance

In [ ]:
%%time
sg_filter = dict(Filters=[{"Name": "tag:Name", "Values": ["allow-inbound-ssh"]}])
ec2_instance_filter = dict(Filters=[{"Name": "tag:Name", "Values": ["my-ec2-instance"]}])
ec2h.attach_sg_to_ec2_instance(sg_filter, ec2_instance_filter, 0, 0, aws_region)